In [17]:
import sys
import os
import configparser
import spotipy
import spotipy.util as util
import spotipy.oauth2 as oauth2
import requests
import json
import config
import time
import pandas as pd
import numpy as np
from json.decoder import JSONDecodeError
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=config.client_id, 
                                                          client_secret=
                                                          config.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [185]:
# get song ids, given a playlist ID, username and the length of the playlist
def get_song_ids(playlistID, playlist_length):
    song_ids = []

    # need to paginate (only 100 ids given per request)
    for i in range(0, playlist_length, 100):
        song_ids.extend([x['track']['id'] for x in 
                       sp.user_playlist_tracks(config.username, 
                                               playlistID, offset=i, 
                                               fields='items(track(id))')['items']])
        time.sleep(.35)
    return song_ids

In [130]:
# get song features, given list of ids to get the features of
def get_features(ids):
    features = []
    i=0
    for track_id_list in ids:
    # can only get 50 at a time
        while i < len(ids):
            features.extend(sp.audio_features(tracks=track_id_list[i:i+50]))
            i += 50
            time.sleep(.35)

        # above will only run if len(ids) > 50
        if len(ids) <= 50:
            features.extend(sp.audio_features(tracks=track_id_list))

    return features

In [187]:
def get_track_objects(track_ids):
    tracks_info = []
    for track_id_list in track_ids:
        info = sp.tracks(track_id_list)
        tracks_info.append(info)
    return tracks_info

In [188]:
#Create a function to collect the tracks info for a given list of ids
def get_track_objects(track_ids):
    tracks_info = []
    for track_id_list in track_ids:
        info = sp.tracks(track_id_list)
        tracks_info.append(info)
    return tracks_info

In [186]:
#Use the artist and track to find a song id
def get_track_id(artist_name,track_name):
    results = sp.search("artist: " + artist_name + ", track: " + track_name)
    return results['tracks']['items'][0]['uri']

In [172]:
def get_artist_id(artist_name):
    results = sp.search("artist:"+artist_name)
    artist_id = results['tracks']['items'][0]['artists'][0]['uri']
    return artist_id

In [273]:
def get_albums_id(artist_id, limit=50):
    albums_ids = []
    for artist_  in artist_id:
        albums = sp.artist_albums(artist_,country="US", limit=limit)
        album_id = [album['uri'].strip('spotify:album:') for album in albums['items']]
        albums_ids.extend(album_id)
    print(albums_ids)
    return album_ids

In [174]:
mykle_lowry = '0JmShn6UeNuPx16P5CmRMQ'

In [274]:
lst = [fergie_baby, mykle_lowry]

In [331]:
albums = sp.artist_albums(fergie_baby,country="US", limit=50)
album_ids = [album['uri'].strip('spotify:album:') for album in albums['items']]

In [307]:
albums_ids = get_albums_id(lst, 50)

['00xvtlRuMv0PVNmuc0WgB8', '7fAPTIRoEp7ZKjU3paNgBK', '4h59IBRhc1vFa6egR8N2XU', '3Md8MLrjVF8TBE1HRZ6or2', '6mNskfgdELss3okGXPBj6C', '0JS0Yq79Jg30gAFy7RaWbk', '0NU0jRCXAr343DtGWTlJCF', '1UTiQHKwQRlK59KKCuVCdw', '2Pv3kgX3wFEXk5LLLSCzE', '1ETECVibld1abNYqKkJNW', '6PFdoOc8LiwJxV6X6jyGS5', '606jg2kXuvBb2ivlQc3B8g', '3qC1IaQlmg5DU4TgYf4gQI', '73rnL4a3Lw5AlqkyvD2jcY']


In [334]:
all_tracks = []
for album_id in album_ids:
    try: 
        tracks = sp.album_tracks(album_id, limit=50)
        all_tracks.append(tracks)
    except:
        print(album_id)

2Pv3kgX3wFEXk5LLLSCzE
1ETECVibld1abNYqKkJNW


In [333]:
len(all_tracks)

10

In [311]:
track_ids = []
for tracks in all_tracks:
    album_tracks = []
    for track in tracks.get('items'):
        album_tracks.append(track.get('uri'))
    track_ids.append(album_tracks)

In [313]:
len(track_ids)

10

In [293]:
track_objects = []
for track_id_list in track_ids:
    tracks = sp.tracks(track_id_list)
    track_objects.append(tracks)

In [314]:
len(track_objects)

10

In [315]:
audio_feature_objects = []
for track_id_list in track_ids:
    features = sp.audio_features(track_id_list)
    audio_feature_objects.append(features)

In [316]:
len(audio_feature_objects)

10

In [319]:
track_features = pd.DataFrame(get_features(track_ids))

In [321]:
spotify_data = {
    "audio_features": audio_feature_objects,
    "tracks": track_objects}

In [328]:
album_features

[{'danceability': 0.815,
  'energy': 0.583,
  'key': 2,
  'loudness': -5.708,
  'mode': 1,
  'speechiness': 0.49,
  'acousticness': 0.191,
  'instrumentalness': 0,
  'liveness': 0.112,
  'valence': 0.357,
  'tempo': 146.1,
  'type': 'audio_features',
  'id': '1nN1pkVuHsJIF8mwUV8ucF',
  'uri': 'spotify:track:1nN1pkVuHsJIF8mwUV8ucF',
  'track_href': 'https://api.spotify.com/v1/tracks/1nN1pkVuHsJIF8mwUV8ucF',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1nN1pkVuHsJIF8mwUV8ucF',
  'duration_ms': 178469,
  'time_signature': 4}]

In [297]:
#Reformatting from a dictionary to a table
df = pd.DataFrame(columns=[
    'name',
    'duration_ms',
    'popularity',
    'num_markets',
    'album',
    'disc_number',
    'is_explicit',
    'track_number',
    'release_date',
    'artist',
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'time_signature',
])

for album_info, album_features in zip(
        spotify_data.get('tracks'), 
        spotify_data.get('audio_features')
        ):
    for track_info, track_features in zip(
        album_info.get('tracks'),
        album_features
        ):
        y = {
            'name': track_info['name'],
            'duration_ms': track_info['duration_ms'],
            'popularity': track_info['popularity'],
            'num_markets': len(track_info['available_markets']),
            'album': track_info['album']['name'],
            'disc_number': track_info['disc_number'],
            'is_explicit': track_info['explicit'],
            'track_number': track_info['track_number'],
            'release_date': track_info['album']['release_date'],
            'artist': track_info['artists'][0]['name'],
            'danceability': track_features['danceability'],
            'energy': track_features['energy'],
            'key': track_features['key'],
            'loudness': track_features['loudness'],
            'mode': track_features['mode'],
            'speechiness': track_features['speechiness'],
            'acousticness': track_features['acousticness'],
            'instrumentalness': track_features['instrumentalness'],
            'liveness': track_features['liveness'],
            'valence': track_features['valence'],
            'tempo': track_features['tempo'],
            'time_signature': track_features['time_signature'],
        }
        df = df.append(y, ignore_index=True)

In [318]:
len(df)

44